### Imports

In [250]:
from analysis_helpers import *
import pandas as pd
import numpy as np
import seaborn as sb
import ffmpeg
import pprint
import math
import os
from math import atan2, degrees

### Load data

In [251]:
# load compiled data from all experiments
data_1 = pd.read_csv('exps1-6_compiled_data.csv')
data_2 = pd.read_csv('exp7_compiled_data.csv')
data_3 = pd.read_csv('exp8_compiled_data.csv')

all_data = pd.concat([data_1, data_2, data_3])

/var/folders/59/tcvs1l0j4ld27ynbn58_v9hh0000gn/T/ipykernel_91164/1694146236.py:2: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data_1 = pd.read_csv('exps1-6_compiled_data.csv')


### Calculate the size of video stimuli

In [301]:
# load a random video from the stimulus sets
probe  = ffmpeg.probe('/Users/kirstenziman/Desktop/Desktop/DIDEC_entire_corpus/analysis_3/subject_8/285896.bmp_pp92_L3_V1_veridical_freeview_isoFalse_fps144.mp4')
video_streams = [stream for stream in probe["streams"] if stream["codec_type"] == "video"]
width = video_streams[0]['coded_width']
height = video_streams[0]['coded_height']

# obtain the raw size of the video and the height:length ratio
print ()
print ('The coded video stimulus size is '+str(height)+' x '+str(width))
print ('The height:length ratio is '+str(int(height/math.gcd(width, height)))+':'+str(int(length/math.gcd(width, height)))); print()

# presentation size on the screen, given size ratio and height in pixels
pixel_size = 500 # presentation height coded in experiment
print ('The presentation size on the screen is a height of 500 pixels')
print ('Given the ratio, that means '+str(pixel_size)+' x '+str(int((pixel_size/(height/math.gcd(width, height)))*length/math.gcd(width, height)))+' pixels in size')

# given the specs of our monitor, calculate dimensions in centimeters
# screen specs --> https://www.displayspecifications.com/en/model/31826fe

# screen dimensions in pixels: 2560 x 1440
# screen dimensions in centimeters: 59.6736 x 33.5664
pixels_per_cm = 2560/59.6736
height_cm = pixel_size / pixels_per_cm
length_cm = 800 / pixels_per_cm

# calculate degrees visual angle of video stimulus
viewing_distance_cm = 50
height_vis_angle = degrees(math.atan2((height_cm/2),viewing_distance_cm))
length_vis_angle = degrees(math.atan2((length_cm/2),viewing_distance_cm))

print()
print('The height in degrees visual angle is '+str(round(height_vis_angle*2,3)))
print('The length in degrees visual angle is '+str(round(length_vis_angle*2,3)))

# for a review on calculating visual angle of a stimulus, see:
# https://osdoc.cogsci.nl/2.9.1/miscellaneous/visual-angle/
# https://en.wikipedia.org/wiki/Visual_angle


The coded video stimulus size is 1050 x 1680
The height:length ratio is 5:8

The presentation size on the screen is a height of 500 pixels
Given the ratio, that means 500 x 800 pixels in size

The height in degrees visual angle is 13.296
The length in degrees visual angle is 21.126


### Average experiment duration

In [211]:
# all_data['responses'] = all_data['responses'].fillna('b')
# all_data[all_data['responses'].str.contains('Q0')]['trial_index']-2

In [212]:
# get times when subjects start reading instructions
start_times = list(all_data[all_data['trial_index']==1]['time_elapsed'])

# get times when subjects enter last response
# note, different ending index for different experiments
end_times = []

for sub in all_data['ID'].unique():
    
    # for exps where subjects enter 'judgment' and 'confidence' (exps 1-6) 
    # get the timestamp of the last 'confidence' rating     
    sub_dat  = all_data[all_data['ID']==sub]
    sub_dat['responses'] = sub_dat['responses'].fillna('nope')
    sub_dat['stimulus'] = sub_dat['stimulus'].fillna('nope')
    
    if sub_dat['version'].unique() in [
       'Mismatched Image', 
       'Scrambled Order versus Veridical (Image)',
       'Scrambled Order versus Veridical (Black)',
       'Horizontal Flip versus Veridical (Black)',
       'Reverse Order versus Veridical (Image)',
       'Reverse Order versus Veridical (Black)']:
            
        sub_conf = sub_dat[sub_dat['responses'].str.contains('confidence')]
        max_t = sub_conf.iloc[69]['time_elapsed'].item()

    # for exps where subjects pressed a button (exp 7)
    # get timestamp of last button press
    elif sub_dat['version'].unique() in ['learning']:
        sub_keys = sub_dat[sub_dat['stimulus'].str.contains('attention')]
        max_t = sub_keys.iloc[69]['time_elapsed'].item()
            
    # for exps where subjects answer multiple questions (exp 8)
    # get the timestamp of the last question response   
    elif sub_dat['version'].unique() in ['Subjective Ratings Over Image']:
        sub_post = sub_dat[sub_dat['responses'].str.contains('post-5')]
        max_t = sub_post.iloc[69]['time_elapsed'].item()
        
    else:
        print('Experiment not recognized')
        
    end_times.append(max_t)

/var/folders/59/tcvs1l0j4ld27ynbn58_v9hh0000gn/T/ipykernel_91164/3270910561.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_dat['responses'] = sub_dat['responses'].fillna('nope')
/var/folders/59/tcvs1l0j4ld27ynbn58_v9hh0000gn/T/ipykernel_91164/3270910561.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_dat['stimulus'] = sub_dat['stimulus'].fillna('nope')


In [164]:
timing_df = pd.DataFrame({'end_times':end_times,'start_times': start_times})
timing_df['timing_diff'] = timing_df['end_times'] - timing_df['start_times']
timing_df['timing_diff_minutes'] = timing_df['timing_diff']/60_000

In [197]:
percent_under_thirty = timing_df[timing_df['timing_diff_minutes']<30].shape[0] / timing_df.shape[0]
print(str(percent_under_thirty*100)+'% of participants finished in under 30 minutes')

93.125% of participants finished in under 30 minutes


### Average frames per second in initial video stimuli

In [1]:
# # calculate average number of video frames in non fps144 videos

# stim_folders = ['/Users/kirstenziman/Desktop/predicting_attention/experiments/experiment_1/STIM_SET_1',
#                '/Users/kirstenziman/Desktop/predicting_attention/experiments/experiment_2/STIM_SET_2',
#                 '/Users/kirstenziman/Desktop/predicting_attention/experiments/experiment_3/STIM_SET_3',
#                 '/Users/kirstenziman/Desktop/predicting_attention/experiments/experiment_4/FOR_REV_BLOBBY_STIM',
#                 '/Users/kirstenziman/Desktop/predicting_attention/experiments/experiment_5/FOR_REV_MAP_BLOBBY_STIM',
#                 '/Users/kirstenziman/Desktop/predicting_attention/experiments/experiment_6/STIM_SET_5',
#                ]

# s = '/Users/kirstenziman/Desktop/predicting_attention/experiments/experiment_7/experiments/'

# for r in range(1,21):
#     d = s+'STIM_SET_'+str(r)
#     stim_folders.append(d)

In [2]:
# for folder in stim_folders:
#     raw_vids = [x for x in os.listdir(folder) if 'mp4' in x and 'fps' not in x]
    
#     print(folder)
#     print(len(raw_vids))
#     print()
    

In [3]:
#raw_vids